In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *
from fastai import *

import matplotlib.pyplot as plt

plt.style.use('ggplot')
PATH = Path('/kaggle/input/plant-pathology-2021-fgvc8/')

In [ ]:
df = pd.read_csv(PATH/'train.csv')
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df.labels.value_counts().plot(kind='bar', figsize=(16,6));

In [ ]:
# Count files in train directory & test directory
!ls ../input/plant-pathology-2021-fgvc8/train_images/ | wc -l
!ls ../input/plant-pathology-2021-fgvc8/test_images/ | wc -l

In [ ]:
PATH

In [ ]:
path= '../input/plant-pathology-2021-fgvc8/'

In [ ]:
# High-Level API
# dls = ImageDataLoaders.from_df(df, path, folder='train_images', label_delim=' ',
#                                item_tfms=Resize(460), batch_tfms=aug_transforms(size=224))

# # DataBlock
# dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
#                    get_x=ColReader('image', pref=str(path+'train_images') + os.path.sep),
#                    get_y=ColReader('labels', label_delim=' '),
#                    item_tfms = Resize(16),
#                    batch_tfms=aug_transforms(mult=2, size=128, flip_vert=True))

# dls = dblock.dataloaders(df)

In [ ]:
# ImageDataLoaders.from_df??
# dls.valid.show_batch()

In [ ]:
# def get_dls(bs, size):
#     dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
#                    get_items=get_image_files,
#                    get_y=parent_label,
#                    item_tfms=Resize(128),
#                    batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
#                                Normalize.from_stats(*imagenet_stats)])
#     return dblock.dataloaders(path, bs=bs)

In [ ]:
# cnn_learner?

In [ ]:
# learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.5))
# learn.lr_find()

# dls = get_dls(64, 128)

# model = xresnet50()
# learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
# learn.fit_one_cycle(5, 3e-3)

# learn = cnn_learner(dls, resnet50, metrics=F1ScoreMulti, loss_func=BCEWithLogitsLossFlat())
# learn.fine_tune(1,0.001)

# # learn.fit_one_cycle(1, 0.01)
# = Path('/kaggle/input/model4/')
#learn = load_learner(MODEL_PATH/'serial_learner_export_3.pkl')
MODEL_PATH = Path('/kaggle/input/model4/')
learn = load_learner(MODEL_PATH/'serial_learner_export_3.pkl')

In [ ]:
# The way to load learners
# learn.model_dir='../input/model' 
# learn.model_dir='/kaggle/working'
# # # Saving serializing
# learn.export('/kaggle/working/serial_learner_export_4.pkl')
# # # loading learner
# learn = load_learner('/kaggle/working/serial_learner_export_3.pkl')

In [ ]:
# Checking loaded model with trained model

# print(f'{learn.dls.vocab}')
# print(f'{export_learner.dls.vocab}')

In [ ]:
# Potentially used in prediciting images work

# test_dl = dls.test_dl(get_image_files('../input/plant-pathology-2021-fgvc8/test_images').sorted())
# learn.get_preds(dl=test_dl)
# export_learner.model.cpu()

# With imported model

# Had an error around Tensorfloats and cuda.Tensorfloats
# This was because the data was trained on GPU and was trying to run on CPU 

# learn.model.cpu()
learn.model.cpu()
dls = learn.dls

# Used in prediciting images
test_dl = dls.test_dl(get_image_files('../input/plant-pathology-2021-fgvc8/test_images')
                          .sorted()
                         )

learn.get_preds(dl=test_dl)

In [ ]:
learn.dls.vocab

In [ ]:
learn.predict(PATH/'test_images/85f8cb619c66b863.jpg')

In [ ]:
learn.predict(PATH/'test_images/ad8770db05586b59.jpg')

In [ ]:
learn.predict(PATH/'test_images/c7b03e718489f3ca.jpg')

In [ ]:
preds_2 = learn.get_preds(dl=test_dl)
arr = preds_2[0].numpy()

In [ ]:
df = pd.DataFrame(data=arr, 
                index=pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')['image'], 
                columns=['cider_apple_rust', 'complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'])


df['labels'] = df[['cider_apple_rust', 'complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']].idxmax(axis=1)

df.drop(columns=['cider_apple_rust', 'complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'], inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('submission.csv')